In [8]:
from datasets import maps
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style

tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers
plt.style.use(astropy_mpl_style)


data_dir  = "/net/diva/scratch-ssd1/mhuertas/users.flatironinstitute.org/~fvillaescusa/priv/DEPnzxoWlaTQ6CjrXqsm0vYi8L7Jy/CMD/2D_maps/data"

In [30]:
negloglik = lambda y, p_y: -p_y.log_prob(y)

def build_model(nfilters, num_components, input_shape, output_shape):
    cnn = tfk.Sequential([
        tfkl.Conv2D(
            nfilters, (4, 4),
            input_shape=(input_shape, input_shape, 1),
            padding="same",
            activation='relu'),
        tfkl.BatchNormalization(),
        tfkl.MaxPool2D((2, 2), strides=2),
        tfkl.Conv2D(
            nfilters * 2, (3, 3),
            padding="same",
            activation='relu'),
        tfkl.MaxPool2D((2, 2), strides=2),
        tfkl.Conv2D(
            nfilters * 4, (2, 2),
            padding="same",
            activation='relu'),
        tfkl.MaxPool2D((2, 2), strides=2),

        tf.keras.layers.Flatten(),
        tfkl.Dense(128, activation='relu'),
        tfkl.Droput(0.5),
        tfkl.Dense(64, activation='relu'),
        tfkl.Droput(0.5),
        tfkl.Dense(64, activation='tanh'),
        tfkl.Dense(tfpl.MixtureNormal.params_size(num_components), activation=None),
        tfpl.MixtureNormal(num_components)
    ])

    cnn.compile(optimizer=tf.optimizers.Adam(learning_rate=0.00002), loss=negloglik)

    return cnn

def preprocessing(example):
#     def conv(x):
    #example['flux'] = to_mag(example['flux'])
    #example['flux'] = example['flux']/46.
    #example = example[np.where(example['mass']>0.5)]
    return example['Mgas'], example['omega_m']

def input_fn(mode='train', batch_size=64):
    """
    mode: 'train' or 'test' or 'val'
    """
    if mode == 'train':
        dataset = tfds.load('maps', split='train[:80%]',data_dir=data_dir)
        dataset = dataset.repeat()
        dataset = dataset.shuffle(10000)
        
    elif mode == 'val':
        dataset = tfds.load('maps', split='train[80%:90%]',data_dir=data_dir)
    else:
        dataset = tfds.load('maps', split='train[90%:]',data_dir=data_dir)

    dataset = dataset.batch(batch_size, drop_remainder=True)
    #dataset = dataset.map(lambda ex: ex['Mgas'], ex['omega_m'])
    dataset = dataset.map(preprocessing) # Apply data preprocessing
    dataset = dataset.prefetch(-1)  # fetch next batches while training current one (-1 for autotune)
    return dataset

In [31]:
dset = input_fn()
dval = input_fn(mode='val')
cnn = build_model(16,1,256,1)
history = cnn.fit(dset, epochs=20,steps_per_epoch=234,validation_data=dval)

Epoch 1/20
234/234 [==============================] - 17s 60ms/step - loss: -0.2246 - val_loss: -0.5470
Epoch 2/20
234/234 [==============================] - 14s 59ms/step - loss: -0.7692 - val_loss: -0.6186
Epoch 3/20
234/234 [==============================] - 14s 59ms/step - loss: -0.9514 - val_loss: -0.5657
Epoch 4/20
234/234 [==============================] - 14s 59ms/step - loss: -1.0897 - val_loss: -0.4231
Epoch 5/20
234/234 [==============================] - 14s 59ms/step - loss: -1.2380 - val_loss: -0.0080
Epoch 6/20
234/234 [==============================] - 14s 59ms/step - loss: -1.3496 - val_loss: 0.0076
Epoch 7/20
234/234 [==============================] - 14s 60ms/step - loss: -1.4709 - val_loss: 0.3470
Epoch 8/20
234/234 [==============================] - 14s 60ms/step - loss: -1.5743 - val_loss: 0.6462
Epoch 9/20
234/234 [==============================] - 14s 59ms/step - loss: -1.6721 - val_loss: 1.3492
Epoch 10/20
234/234 [==============================] - 14s 60ms/step